In [3]:
import pandas as pd
import numpy as np
import pickle as pkl
import tensorflow as tf
import pickle as pkl
import math
import re
from sklearn.model_selection import train_test_split
from collections import defaultdict
from gensim.models import Word2Vec
from scipy.sparse import csr_matrix
from keras import backend as K
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer, one_hot
from keras.callbacks import ModelCheckpoint
from keras.models import Model, load_model, Sequential
from keras.layers import Input, Activation, Dot, LSTM, Dense, Embedding, Bidirectional, Lambda, Flatten, RepeatVector, Reshape, Permute, Concatenate, TimeDistributed
from keras.utils import to_categorical
from time import time

In [4]:
#### parameter setting

train = pd.read_csv('data/all/train.csv', header=None)
test = pd.read_csv('data/all/test.csv', header=None)
vocab_size = 30000
latent_dim = 256
embedding_dim = 256
batch_size=128
epochs = 20
keep_prob = 0.5
input_max_length = 69
output_max_length = 25
attention = None


In [5]:
#### function definition

def to_sequence(word_array, vocab):
    return list(map(lambda word: vocab[word] if vocab[word] <= vocab_size else vocab['unk'] , word_array))
def to_sentence(sequence, reverse_vocab):
    if vocab['EOS'] in sequence: length = np.where(sequence == vocab['EOS'])[0][0]
    return list(map(lambda index: reverse_vocab[index], sequence[:length]))

In [7]:

texts = list(train[0].values)
texts += list(train[1].values)
texts += list(test[0].values)
texts = list(map(lambda x: x.split(' '), texts))

vocab_freq = defaultdict(int)
vocab = None
reverse_vocab = None

for row in texts:
    for word in row:
        vocab_freq[word] += 1
        
vocab = sorted(vocab_freq.items(), key=lambda kv: kv[1], reverse=True)
vocab = dict([('unk', -1)] + vocab)
cnt = 1
for key, value in vocab.items():
    vocab[key] = cnt
    cnt += 1
reverse_vocab = dict([(v, k) for k, v in vocab.items()])

In [7]:
# model = Word2Vec(texts, size=256, window=5, min_count=5, workers=4)
# model.save('model/w2v')

In [10]:
wv_model = Word2Vec.load('model/w2v')
embeddings_index = {}
for k, v in wv_model.wv.vocab.items():
    embeddings_index[k] = wv_model.wv[k]
embedding_matrix = np.zeros(shape=(vocab_size+1, embedding_dim))
for word, i in vocab.items():
    if i > 10000: break
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
    else:
        if word == 'unk':
            embedding_matrix[i] = np.average(list(embeddings_index.values()), axis=0)
        else:
            print(word, i)

In [160]:
tf.reset_default_graph()
graph = tf.Graph()

with graph.as_default():
    encoder_inputs = tf.placeholder( tf.int32, shape=(None, input_max_length), name='encoder_inputs')
    decoder_inputs = tf.placeholder( tf.int32, shape=(None, output_max_length), name='decoder_inputs')
    decoder_outputs = tf.placeholder( tf.int32, shape=(None, output_max_length), name='decoder_outputs')

    source_sequence_length = tf.placeholder(tf.int32, shape=(None, ), name='encoder_lengths')
    decoder_lengths = tf.placeholder(tf.int32, shape=(None,), name='decoder_lengths')

    embeddings = tf.get_variable(
        "embeddings", [vocab_size+1, embedding_dim]
    )

    embedding_placeholder = tf.placeholder(tf.float32, [vocab_size+1, embedding_dim])
    embedding_init = embeddings.assign(embedding_placeholder)
    encoder_emb_inp = tf.nn.embedding_lookup(embeddings, encoder_inputs)
    decoder_emb_inp = tf.nn.embedding_lookup(embeddings, decoder_inputs)

    with tf.name_scope("encoder_layer"):
        # Build RNN cell
        forward_cell = tf.contrib.rnn.LSTMCell(latent_dim, name='forward_cell')
        backward_cell = tf.contrib.rnn.LSTMCell(latent_dim, name='backward_cell')
        forward_cell = tf.contrib.rnn.DropoutWrapper(forward_cell, output_keep_prob = keep_prob)
        backward_cell = tf.contrib.rnn.DropoutWrapper(backward_cell, output_keep_prob = keep_prob)
        
        bi_outputs, bi_state = tf.nn.bidirectional_dynamic_rnn(
            forward_cell,
            backward_cell,
            inputs = encoder_emb_inp,
            sequence_length = source_sequence_length,
            time_major = False,
            dtype = tf.float32
        )
        c = tf.concat([bi_state[0].c, bi_state[1].c], axis=1)
        h = tf.concat([bi_state[0].h, bi_state[1].h], axis=1)
        tf.summary.histogram('encoder_c', c)
        tf.summary.histogram('encoder_h', h)
        
        encoder_outputs = tf.concat(bi_outputs, 2)
        encoder_state = tf.contrib.rnn.LSTMStateTuple(c, h)


    with tf.variable_scope('decoder_layer') as decoder_scope:
        decoder_cell = tf.contrib.rnn.LSTMCell(latent_dim * 2, name='decoder_cell')
        decoder_cell = tf.contrib.rnn.DropoutWrapper(decoder_cell, output_keep_prob = keep_prob)
        projection_layer = tf.layers.Dense(vocab_size, use_bias=False)
        
        if attention == 'bahdanau':
            bahdanau_attention = tf.contrib.seq2seq.BahdanauAttention(
                latent_dim * 2,
                encoder_outputs,
                memory_sequence_length= decoder_lengths,
                normalize = True
            )
            decoder_cell = tf.contrib.seq2seq.AttentionWrapper(
                decoder_cell,
                bahdanau_attention,
                alignment_history = True,
                attention_layer_size = latent_dim * 2,
                name = "attention"
            )
            decoder_initial_state = decoder_cell.zero_state(batch_size, tf.float32).clone(cell_state=encoder_state)
        elif attention == 'luong':
            luong_attention = tf.contrib.seq2seq.LuongAttention(
                latent_dim * 2,
                encoder_outputs,
                memory_sequence_length= decoder_lengths,
            )
            decoder_cell = tf.contrib.seq2seq.AttentionWrapper(
                decoder_cell,
                luong_attention,
                alignment_history = True,
                attention_layer_size = latent_dim * 2,
                name = "attention"
            )
            decoder_initial_state = decoder_cell.zero_state(batch_size, tf.float32).clone(cell_state=encoder_state)
        else:
            decoder_initial_state = encoder_state

        # Helper
        helper = tf.contrib.seq2seq.TrainingHelper(
            inputs = decoder_emb_inp,
            sequence_length = decoder_lengths,
            time_major = False
        )

        # Decoder
        decoder = tf.contrib.seq2seq.BasicDecoder(
            cell = decoder_cell,
            helper = helper,
            initial_state = decoder_initial_state,
            output_layer = projection_layer
        )
    with tf.name_scope("loss"):
        # Dynamic decoding
        outputs, decoder_state, _ = tf.contrib.seq2seq.dynamic_decode(
            decoder = decoder
        )
        if attention != None:
            tf.summary.histogram('decoder_c', decoder_state.cell_state[0])
            tf.summary.histogram('decoder_h', decoder_state.cell_state[1])
            tf.summary.histogram('attention', decoder_state.attention)
            attention_matrices = decoder_state.alignment_history.stack(name='train_atttention_matrix')
        else:
            tf.summary.histogram('decoder_c', decoder_state.c)
            tf.summary.histogram('decoder_h', decoder_state.h)
        logits = outputs.rnn_output

        crossent = tf.nn.sparse_softmax_cross_entropy_with_logits(
            labels = decoder_outputs,
            logits = logits
        )
        non_zero = tf.cast(tf.not_equal(decoder_outputs, 0), tf.float32)
        train_loss = (tf.reduce_sum(crossent * non_zero) / batch_size)
        tf.summary.scalar('cross_entropy', train_loss)
        
    with tf.name_scope("train"):
        params = tf.trainable_variables()
        gradients = tf.gradients(train_loss, params)
        clipped_gradients, _ = tf.clip_by_global_norm(gradients, 5.0)
        train_op = tf.train.AdamOptimizer(1e-3).apply_gradients(zip(clipped_gradients, params))
        

        
    with tf.name_scope("inference_layer"):
        num_sequences_to_decode = tf.placeholder(shape=(), dtype=tf.int32, name="num_seq")
        start_tokens = tf.tile([vocab['SOS']], [num_sequences_to_decode])
        inference_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(
            embeddings,
            start_tokens,
            vocab['EOS']
        )

        inference_decoder = tf.contrib.seq2seq.BasicDecoder(
            decoder_cell, inference_helper, decoder_initial_state,
            output_layer = projection_layer
        )
        maximum_iterations = tf.round(tf.reduce_max(input_max_length))

        greedy_decoding_result, greedy_final_state, _ = tf.contrib.seq2seq.dynamic_decode(
            inference_decoder, maximum_iterations=maximum_iterations
        )
        if attention != None:
            inference_attention_matrices = greedy_final_state.alignment_history.stack(
                name="inference_attention_matrix")
        translations = greedy_decoding_result.sample_id

    merged_summary = tf.summary.merge_all()
    init = tf.global_variables_initializer()

In [112]:
sess = tf.Session(config=tf.ConfigProto(), graph=graph)
sess.run(init)
sess.run(embedding_init, feed_dict={embedding_placeholder: embedding_matrix})

#### tensorboard
with graph.as_default():
    writer = tf.summary.FileWriter('logdir/model_name')
    writer.add_graph(sess.graph)

In [117]:
#### training process

epochs = 2
shuffle_index = np.array(range(0, len(src_data)))
for epoch in range(epochs):
    for i in range(0, len(src_data), batch_size):
        index_batch = shuffle_index[i:i + batch_size]
        train_out = np.array(list(map(lambda x: x[1:], tar_data[index_batch])))
        train_tar = np.array(list(map(lambda x: x[:-1], tar_data[index_batch])))

        train_src_length = src_lengths[index_batch]
        train_tar_length = np.array([25] * len(index_batch))
        
        train_src = np.array(pad_sequences(src_data[index_batch], input_max_length, padding='post'))
        train_tar = np.array(pad_sequences(train_tar, output_max_length, padding='post'))
        train_out = np.array(pad_sequences(train_out, output_max_length, padding='post'))

        feed_dict = {
            encoder_inputs: train_src,
            decoder_inputs: train_tar,
            decoder_outputs: train_out,
            source_sequence_length: train_src_length,
            decoder_lengths: train_tar_length
        }
        if index_batch.shape[0] != batch_size:
            continue
        loss_epoch = sess.run(train_loss, feed_dict)
        if (math.isnan(loss_epoch)):
            print(i)
        else:
            sess.run(train_op, feed_dict)
        if i % 1280 == 0:
            s = sess.run(merged_summary, feed_dict)
            writer.add_summary(s, epoch * len(src_data) + i)
            print("iter {}: loss : {}".format(i, loss_epoch))

    loss_epoch = sess.run(train_loss, feed_dict)
    print("epcoh {}: loss : {}".format(epoch, loss_epoch))

iter 0: loss : 19.907690048217773
iter 1280: loss : 20.95264434814453
iter 2560: loss : 20.716644287109375
iter 3840: loss : 18.418872833251953
iter 5120: loss : 33.53544616699219
iter 6400: loss : 19.420324325561523
iter 7680: loss : 23.82167625427246
iter 8960: loss : 22.351322174072266
iter 10240: loss : 21.43043327331543
iter 11520: loss : 21.610748291015625
iter 12800: loss : 23.894025802612305
iter 14080: loss : 20.366056442260742
iter 15360: loss : 18.024038314819336
iter 16640: loss : 23.31865119934082
iter 17920: loss : 20.91519546508789
iter 19200: loss : 19.04947853088379
iter 20480: loss : 21.345624923706055
iter 21760: loss : 21.700475692749023
iter 23040: loss : 24.240285873413086
iter 24320: loss : 30.5662899017334
iter 25600: loss : 25.48529052734375
iter 26880: loss : 23.891647338867188
iter 28160: loss : 20.30861473083496
iter 29440: loss : 18.400169372558594
iter 30720: loss : 20.843971252441406
iter 32000: loss : 29.648603439331055
iter 33280: loss : 20.395748138427

iter 275200: loss : 25.865367889404297
iter 276480: loss : 24.958187103271484
iter 277760: loss : 18.680038452148438
iter 279040: loss : 25.594371795654297
iter 280320: loss : 18.2983455657959
iter 281600: loss : 20.347461700439453
iter 282880: loss : 24.58647346496582
iter 284160: loss : 22.84800148010254
iter 285440: loss : 20.63949966430664
iter 286720: loss : 23.919050216674805
iter 288000: loss : 20.526084899902344
iter 289280: loss : 17.681901931762695
iter 290560: loss : 25.131240844726562
iter 291840: loss : 15.68746280670166
iter 293120: loss : 22.05475616455078
iter 294400: loss : 24.686447143554688
iter 295680: loss : 20.526996612548828
iter 296960: loss : 20.086393356323242
iter 298240: loss : 17.09345817565918
iter 299520: loss : 24.50650405883789
iter 300800: loss : 15.113102912902832
iter 302080: loss : 17.177833557128906
iter 303360: loss : 20.61672592163086
iter 304640: loss : 18.851512908935547
iter 305920: loss : 25.729209899902344
iter 307200: loss : 22.278490066528

iter 547840: loss : 16.01066017150879
iter 549120: loss : 14.277397155761719
iter 550400: loss : 20.370216369628906
iter 551680: loss : 15.227815628051758
iter 552960: loss : 19.64551544189453
iter 554240: loss : 22.293949127197266
iter 555520: loss : 18.676189422607422
iter 556800: loss : 15.606348037719727
iter 558080: loss : 24.892988204956055
iter 559360: loss : 20.186954498291016
iter 560640: loss : 20.98318099975586
iter 561920: loss : 17.635404586791992
iter 563200: loss : 18.327362060546875
iter 564480: loss : 15.868371963500977
iter 565760: loss : 22.697500228881836
iter 567040: loss : 24.749649047851562
iter 568320: loss : 18.068479537963867
iter 569600: loss : 21.01357078552246
iter 570880: loss : 20.845458984375
iter 572160: loss : 23.69152069091797
iter 573440: loss : 21.76560401916504
iter 574720: loss : 15.020743370056152
iter 576000: loss : 28.323301315307617
iter 577280: loss : 19.20453643798828
iter 578560: loss : 17.44856071472168
iter 579840: loss : 17.1615505218505

InvalidArgumentError: ConcatOp : Dimensions of inputs should match: shape[0] = [6,256] vs. shape[1] = [128,512]
	 [[Node: loss/decoder/while/BasicDecoderStep/decoder/attention/concat = ConcatV2[N=2, T=DT_FLOAT, Tidx=DT_INT32, _device="/job:localhost/replica:0/task:0/device:GPU:0"](loss/decoder/while/Switch_10:1, loss/decoder/while/Switch_5:1, loss/decoder/while/BasicDecoderStep/decoder/attention/concat/axis)]]
	 [[Node: loss/decoder/while/BasicDecoderStep/TrainingHelperNextInputs/add/_48 = _Send[T=DT_INT32, client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device_incarnation=1, tensor_name="edge_107_loss/decoder/while/BasicDecoderStep/TrainingHelperNextInputs/add", _device="/job:localhost/replica:0/task:0/device:CPU:0"](loss/decoder/while/BasicDecoderStep/TrainingHelperNextInputs/add)]]

Caused by op 'loss/decoder/while/BasicDecoderStep/decoder/attention/concat', defined at:
  File "c:\u

In [120]:
#### model saver

with graph.as_default():
    saver = tf.train.Saver()
    save_path = saver.save(sess, "model/model.ckpt")
    print("Model saved in path: %s" % save_path)

Model saved in path: model/luong_attention_4.ckpt


In [121]:
#### testing

test_data = list(map(lambda x: to_sequence(x.split(' '), vocab), test[0].values))
test_lengths = list(map(lambda x: len(x), test_data))

with open('output.csv', 'w', encoding='utf-8') as f,tf.Session(config=tf.ConfigProto(), graph=graph) as sess:
    saver = tf.train.Saver()    
    saver.restore(sess, "model/model.ckpt")
    print("Model restored.")
    
    for i in range(0, len(test_data), batch_size):
        test_data_batch = np.array(pad_sequences(test_data[ i: i + batch_size ], input_max_length, padding='post'))
        test_lengths_batch =  test_lengths[ i: i + batch_size ]
        test_batch_size = len(test_lengths_batch)
        
        if test_batch_size == 0:
            continue
        elif test_batch_size != batch_size:
            padding = np.zeros(shape=(batch_size - test_batch_size, input_max_length), dtype=np.int32)
            test_data_batch = np.concatenate((test_data_batch, padding), axis=0)
            padding = np.zeros(shape=(batch_size - test_batch_size), dtype=np.int32)
            test_lengths_batch = np.concatenate((test_lengths_batch, padding), axis=0)
                               
            feed_dict = {
                encoder_inputs: test_data_batch,
                source_sequence_length: test_lengths_batch,
                decoder_lengths: np.array([25] * batch_size),
                num_sequences_to_decode: batch_size
            }
            replies = sess.run(translations, feed_dict=feed_dict)[:test_batch_size, :]
        else:
            feed_dict = {
                encoder_inputs: test_data_batch,
                source_sequence_length: test_lengths_batch,
                decoder_lengths: np.array([25] * batch_size),
                num_sequences_to_decode: batch_size
            }
            replies = sess.run(translations, feed_dict=feed_dict)
        replies = list(map(lambda x: to_sentence(x, reverse_vocab), replies))
        replies = list(map(lambda x: ' '.join(x), replies))
        for result in replies:
            f.write('{}\n'.format(result))

INFO:tensorflow:Restoring parameters from model/luong_attention_4.ckpt


INFO:tensorflow:Restoring parameters from model/luong_attention_4.ckpt


Model restored.
